In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [ ]:
df = pd.read_csv('explored_existed_audible.csv')

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df.drop(columns='Unnamed: 0')

In [ ]:
df_100 = df.head(100)

## Get original date

In [ ]:
def get_original_publication_date(book_name, author_name):
    # Create a search URL for Goodreads
    base_url = "https://www.goodreads.com/search?utf8=%E2%9C%93&search_type=books&search%5Bfield%5D=on&q="
    search_url = base_url + book_name.replace(" ", "+")
    
    # Fetch the search results page
    response = requests.get(search_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all book titles
    book_entries = soup.select('.tableList .bookTitle')
    
    for entry in book_entries:
        book_title = entry.get_text(strip=True)
        if book_name in book_title:
            book_data = entry.find_parent('tr')
            author = book_data.select_one('.authorName').get_text(strip=True)
            if author_name in author:
                details = book_data.select_one('.greyText').get_text(strip=True)
                details_split = details.split("—")
                try:
                    if len(details_split) > 2:
                        pub_year = details_split[2].strip().split()[1] #get publication date text
                        return pub_year
                except IndexError:
                    # Handle the error or log it
                    print(f"Error processing publication date for {book_name} by {author_name}")
                    return None
                
    return None


In [ ]:
df_100.loc[:, 'original_publication_date'] = None

#iterate through each row and fetch the original publication date
for index, row in df_100.iterrows():
    title = row['title']
    author = row['author']
    original_publication_date = get_original_publication_date(title, author)
    
    # Update the original_publication_date column in the dataframe
    df_100.at[index, 'original_publication_date'] = original_publication_date

In [ ]:
df_100

## Get author url

In [ ]:
def get_author_url(book_name, author_name):
    # Create a search URL for Goodreads
    base_url = "https://www.goodreads.com/search?utf8=%E2%9C%93&search_type=books&search%5Bfield%5D=on&q="
    search_url = base_url + book_name.replace(" ", "+")
    
    # Fetch the search results page
    response = requests.get(search_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all book titles
    book_entries = soup.select('.tableList .bookTitle')
    
    for entry in book_entries:
        book_title = entry.get_text(strip=True)
        if book_name in book_title:
            book_data = entry.find_parent('tr')
            author_tag = book_data.select_one('.authorName')
            author = author_tag.get_text(strip=True)
            if author_name in author:
                author_url = author_tag['href']
                return author_url

    return None

In [ ]:
df_100.loc[:, 'author_url'] = None

#iterate through each row and fetch the author url
for index, row in df_100.iterrows():
    title = row['title']
    author = row['author']
    author_url = get_author_url(title, author)
    
    # Update the author_url in the dataframe
    df_100.at[index, 'author_url'] = author_url

In [ ]:
df_100